In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
RESULT_FILE = '../files/results/experimental_results_CECOVEL.csv'

In [ ]:
results = pd.read_csv(RESULT_FILE, delimiter=";")

# Add column with architecture type (TCN or LSTM)
results['ARCHITECTURE'] = results['MODEL'].map(lambda x: 'LSTM' if 'LSTM' in x else ('TCN' if 'TCN' in x else 'NONE'))

#Set model characteristic as index
index_rows = results[['MODEL', 'MODEL_DESCRIPTION', 'FORECAST_HORIZON', 'PAST_HISTORY', 'BATCH_SIZE']].copy()
results = results.set_index(['MODEL', 'MODEL_DESCRIPTION', 'FORECAST_HORIZON', 'PAST_HISTORY', 'BATCH_SIZE'])

results.head()

In [ ]:
metric_columns = ['mse', 'rmse', 'nrmse', 'mae', 'wape', 'mpe', 'mape',
       'mdape', 'smape', 'smdape', 'mase', 'rmspe', 'rmsse', 'mre', 'rae',
       'mrae', 'std_ae', 'std_ape']
METRIC = 'wape'
cmap={'TCN':(0.5019607843137255, 0.5019607843137255, 0.9916562369628703), 'LSTM':(0.4519420198186802, 0.16470588235294117, 0.32941176470588235)}

In [ ]:
top_by_pasthistory = pd.DataFrame(columns=['PAST_HISTORY','ARCHITECTURE', METRIC])

for ph, _results in results.groupby('PAST_HISTORY'):
    for arch, __results in _results.groupby('ARCHITECTURE'):
        top_by_pasthistory = top_by_pasthistory.append(__results.reset_index().sort_values(METRIC)[['PAST_HISTORY','ARCHITECTURE', METRIC]].head(21), ignore_index=True)

fig, ax = plt.subplots(1,1,figsize=(4.75,3))
sns.boxplot(data=top_by_pasthistory, x='PAST_HISTORY', y=METRIC, hue='ARCHITECTURE', linewidth=0.75, palette=cmap, width=0.5, whis=100, hue_order=['TCN', 'LSTM'], ax=ax)    
ax.set_xlabel('Past history')
ax.set_ylabel(METRIC.upper())
fig.savefig('../files/images/EV-ResultsDistribution-pastHistory.eps'.format(ph), format='eps', bbox_inches='tight')

fig.show()

In [ ]:
top_tcn = results[results['ARCHITECTURE']=='TCN'].sort_values(METRIC).head(1)
print('TCN')
print('\t'+'\n\t'.join(['{}: {}'.format(k,v) for k,v in zip(top_tcn.index.names,top_tcn.index[0])]))
print('\tEPHOCS: '+str(top_tcn[['EPOCHS']].values[0][0]))


top_lstm = results[results['ARCHITECTURE']=='LSTM'].sort_values(METRIC).head(1)
print('LSTM')
print('\t'+'\n\t'.join(['{}: {}'.format(k,v) for k,v in zip(top_lstm.index.names,top_lstm.index[0])]))
print('\tEPHOCS: '+str(top_lstm[['EPOCHS']].values[0][0]))

In [ ]:
top_by_pasthistory = pd.DataFrame(columns=['PAST_HISTORY','ARCHITECTURE', METRIC])

for ph, _results in results.groupby('PAST_HISTORY'):
    for arch, __results in _results.groupby('ARCHITECTURE'):
        top_by_pasthistory = top_by_pasthistory.append(__results.reset_index().sort_values(METRIC)[['PAST_HISTORY','ARCHITECTURE', METRIC]].head(1), ignore_index=True)

sns.barplot(data=top_by_pasthistory, x='PAST_HISTORY', y=METRIC, hue='ARCHITECTURE')
plt.show()

top_by_pasthistory.pivot(index='ARCHITECTURE', columns='PAST_HISTORY', values=METRIC)

In [ ]:
# 25 Epochs
fig,ax = plt.subplots(1,3, figsize=(16,4))

for i, (ph, _results) in enumerate(results[results['EPOCHS']==25].groupby('PAST_HISTORY')):
    
    line_columns = []
    for arch, __results in _results.groupby('ARCHITECTURE'):
        loss = np.fromstring(__results.sort_values(METRIC).head(1)['loss'].iloc[0].replace('[','').replace(']',''), dtype='float32', sep=',')
        val_loss = np.fromstring(__results.sort_values(METRIC).head(1)['val_loss'].iloc[0].replace('[','').replace(']',''), dtype='float32', sep=',')
          
        loss, val_loss = (loss[1:], val_loss[1:]) if i < 2 else (loss[3:], val_loss[3:])
        
        loss_line = ax[i].plot(loss, label=('loss',arch), c=cmap[arch])
        valloss_line = ax[i].plot(val_loss, label=('val',arch), c=cmap[arch], linestyle='--')
        
        line_columns.append(loss_line[0])
        line_columns.append(valloss_line[0])
        
        ax[i].set_xlabel('Epochs')
        if i==0:
            ax[i].set_ylabel('loss')
        ax[i].set_title('Past history '+str(ph))
    
    # Create legend
    leg = ax[i].legend(line_columns, ['', '', '    Training', '    Validation'], 
             title='LSTM   TCN                  ', handletextpad=-0.5,  
             ncol=2, numpoints=1)

fig.suptitle('25 epochs', y=1.025, fontsize=15)
fig.show()

# 50 Epochs
fig,ax = plt.subplots(1,3, figsize=(16,4))

for i, (ph, _results) in enumerate(results[results['EPOCHS']==50].groupby('PAST_HISTORY')):
    
    line_columns = []
    for arch, __results in _results.groupby('ARCHITECTURE'):
        loss = np.fromstring(__results.sort_values(METRIC).head(1)['loss'].iloc[0].replace('[','').replace(']',''), dtype='float32', sep=',')
        val_loss = np.fromstring(__results.sort_values(METRIC).head(1)['val_loss'].iloc[0].replace('[','').replace(']',''), dtype='float32', sep=',')
          
        loss, val_loss = (loss[1:], val_loss[1:]) if i < 2 else (loss[3:], val_loss[3:])
        
        loss_line = ax[i].plot(loss, label=('loss',arch), c=cmap[arch])
        valloss_line = ax[i].plot(val_loss, label=('val',arch), c=cmap[arch], linestyle='--')
        
        line_columns.append(loss_line[0])
        line_columns.append(valloss_line[0])
        
        ax[i].set_xlabel('Epochs')
        if i==0:
            ax[i].set_ylabel('loss')
        ax[i].set_title('Past history '+str(ph))
    
    # Create legend
    leg = ax[i].legend(line_columns, ['', '', '    Training', '    Validation'], 
             title='LSTM   TCN                  ', handletextpad=-0.5,  
             ncol=2, numpoints=1)
fig.suptitle('50 epochs', y=1.025, fontsize=15)
fig.show()

# 100 Epochs
fig,ax = plt.subplots(1,3, figsize=(16,4))

for i, (ph, _results) in enumerate(results[results['EPOCHS']==100].groupby('PAST_HISTORY')):
    
    line_columns = []
    for arch, __results in _results.groupby('ARCHITECTURE'):
        loss = np.fromstring(__results.sort_values(METRIC).head(1)['loss'].iloc[0].replace('[','').replace(']',''), dtype='float32', sep=',')
        val_loss = np.fromstring(__results.sort_values(METRIC).head(1)['val_loss'].iloc[0].replace('[','').replace(']',''), dtype='float32', sep=',')
          
        loss, val_loss = (loss[1:], val_loss[1:]) if i < 2 else (loss[3:], val_loss[3:])
        
        loss_line = ax[i].plot(loss, label=('loss',arch), c=cmap[arch])
        valloss_line = ax[i].plot(val_loss, label=('val',arch), c=cmap[arch], linestyle='--')
        
        line_columns.append(loss_line[0])
        line_columns.append(valloss_line[0])
        
        ax[i].set_xlabel('Epochs')
        if i==0:
            ax[i].set_ylabel('loss')
        ax[i].set_title('Past history '+str(ph))
    
    # Create legend
    leg = ax[i].legend(line_columns, ['', '', '    Training', '    Validation'], 
             title='LSTM   TCN                  ', handletextpad=-0.5,  
             ncol=2, numpoints=1)

fig.suptitle('100 epochs', y=1.025, fontsize=15)
fig.show()


In [ ]:
fig,ax = plt.subplots(1,3, figsize=(16,4))

for i, (ph, _results) in enumerate(results.groupby('PAST_HISTORY')):
    for arch, __results in _results.groupby('ARCHITECTURE'):
        loss = np.fromstring(__results.sort_values(METRIC).head(1)['loss'].iloc[0].replace('[','').replace(']',''), dtype='float32', sep=',')
        val_loss = np.fromstring(__results.sort_values(METRIC).head(1)['val_loss'].iloc[0].replace('[','').replace(']',''), dtype='float32', sep=',')
        
        
        loss, val_loss = (loss[1:], val_loss[1:]) if i < 2 else (loss[3:], val_loss[3:])
        
        ax[i].plot(loss, label=('loss',arch), c=cmap[arch])
        ax[i].plot(val_loss, label=('val',arch), c=cmap[arch], linestyle='--')
        ax[i].set_xlabel('Epochs')
        if i==0:
            ax[i].set_ylabel('loss')
    ax[i].legend()
ax[1].set_title('100 epochs')

fig.show()

In [ ]:
for i, (ph, _results) in enumerate(results[results['EPOCHS']==50].groupby('PAST_HISTORY')):
    fig,ax = plt.subplots(1,1, figsize=(3,2.5))
    line_columns = []
    for arch, __results in _results.groupby('ARCHITECTURE'):
        print(str(i), str(ph),arch) 
        top_tcn = __results.sort_values(METRIC).head(1)
        print('\t'+'\n\t'.join(['{}: {}'.format(k,v) for k,v in zip(top_tcn.index.names,top_tcn.index[0])]))
        print('\tEPHOCS: '+str(top_tcn[['EPOCHS']].values[0][0]))
        loss = np.fromstring(__results.sort_values(METRIC).head(1)['loss'].iloc[0].replace('[','').replace(']',''), dtype='float32', sep=',')
        val_loss = np.fromstring(__results.sort_values(METRIC).head(1)['val_loss'].iloc[0].replace('[','').replace(']',''), dtype='float32', sep=',')
          
        loss, val_loss = (loss[1:], val_loss[1:]) if i < 2 else (loss[3:], val_loss[3:])
        
        loss_line = ax.plot(loss, label=('loss',arch), c=cmap[arch])
        valloss_line = ax.plot(val_loss, label=('val',arch), c=cmap[arch], linestyle='--')
        
        line_columns.append(loss_line[0])
        line_columns.append(valloss_line[0])
        
        ax.set_xlabel('Epochs')
        ax.set_ylabel('Loss (MAE)')
    
    # Create legend
    leg = ax.legend(line_columns, ['', '', '    Training', '    Validation'], 
             title='LSTM   TCN                  ', handletextpad=-0.5,  
             ncol=2, numpoints=1)
    fig.savefig('../files/images/EV-loss_50epochs_{}PastHistory.eps'.format(ph), format='eps', bbox_inches='tight')
